In [20]:
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
#temporary until bugs fixed in current master
!git reset --hard 6b134d93c51c774c09df2c10c228d35fc2201e46
%cd /content

!pip install -r yolov5/requirements.txt  # install dependencies
%cd yolov5

import torch
from IPython.display import Image, clear_output  # to display images
from utils.google_utils import gdrive_download  # to download models/datasets
import cv2
import tqdm
import numpy as np
import glob
import numpy as np
import pandas as pd
from utils.datasets import *
from utils.utils import *
from google.colab.patches import cv2_imshow


clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

Setup complete. Using torch 1.5.1+cu101 CPU


# Detection

In [ ]:
vidcap = cv2.VideoCapture('/content/drive/My Drive/2BE or NOT 2B (Best Car Number Plate Combo in UK).mp4')
success,image = vidcap.read()
count = 0
while success:
  cv2.imwrite("/content/drive/My Drive/output_YOLO_images/frame%d.jpg" % count, image)     # save frame as JPEG file      
  success,image = vidcap.read()
  
  count += 1

In [ ]:
!python detect.py --weights /content/drive/My\ Drive/last_yolov5s_results.pt --img 416 --conf 0.4 --source /content/drive/My\ Drive/output_YOLO_images

Namespace(agnostic_nms=False, augment=False, classes=None, conf_thres=0.4, device='', img_size=416, iou_thres=0.5, output='inference/output', save_txt=False, source='/content/drive/My Drive/output_YOLO_images', update=False, view_img=False, weights='/content/drive/My Drive/last_yolov5s_results.pt')
Using CPU

image 1/1045 /content/drive/My Drive/output_YOLO_images/frame0.jpg: 288x416 Done. (0.196s)
image 2/1045 /content/drive/My Drive/output_YOLO_images/frame1.jpg: 288x416 2 bikes, Done. (0.143s)
image 3/1045 /content/drive/My Drive/output_YOLO_images/frame10.jpg: 288x416 Done. (0.135s)
image 4/1045 /content/drive/My Drive/output_YOLO_images/frame100.jpg: 288x416 1 cars, Done. (0.144s)
image 5/1045 /content/drive/My Drive/output_YOLO_images/frame1000.jpg: 288x416 Done. (0.131s)
image 6/1045 /content/drive/My Drive/output_YOLO_images/frame1001.jpg: 288x416 Done. (0.143s)
image 7/1045 /content/drive/My Drive/output_YOLO_images/frame1002.jpg: 288x416 Done. (0.143s)
image 8/1045 /content/d

In [ ]:
img_array = []
for filename in sorted(glob.glob('/content/yolov5/inference/output/*.jpg')):
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)
    img_array.append(img)
 
 
out = cv2.VideoWriter('/content/drive/My Drive/YOLO_Output/project.avi',cv2.VideoWriter_fourcc(*'DIVX'), 30, size)
 
for i in range(len(img_array)):
    out.write(img_array[i])
out.release()

# Cropping

In [39]:
vs = cv2.VideoCapture('/content/drive/My Drive/2BE or NOT 2B (Best Car Number Plate Combo in UK).mp4')

In [40]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Load model
model = torch.load('/content/drive/My Drive/last_yolov5s_results.pt', map_location=device)['model'].float()  # load to FP32
model.to(device).eval()
pass

In [41]:
labels = pd.DataFrame(columns=['image','label'])
path = '/content/drive/My Drive/Test_1/'

In [42]:
j = 0 
while True:
  (grabbed,frame) = vs.read()
  if not grabbed:
    break
  img = cv2.resize(frame,(416,416))
  img = img[:, :, ::-1].transpose(2, 0, 1)  # BGR to RGB, to 3x416x416
  img = np.ascontiguousarray(img)
  img = torch.from_numpy(img).to(device)
  img =  img.float()  # uint8 to fp16/32
  img /= 255.0   
  if img.ndimension() == 3:
    img = img.unsqueeze(0)
  pred = model(img, augment=False)[0]
  pred = non_max_suppression(pred, 0.5, 0.6)
  boxes = []
  scores = []
  lb = []
  for i, det in enumerate(pred):  # detections per image
    # save_path = 'draw/' + image_id + '.jpg'
    if det is not None and len(det):
        # Rescale boxes from img_size to im0 size
        det[:, :4] = scale_coords(img.shape[2:], det[:, :4], frame.shape).round()

        # Write results
        for *xyxy, conf, cls in det:
            boxes.append([int(xyxy[0]), int(xyxy[1]), int(xyxy[2]), int(xyxy[3])])
            scores.append(conf)
            lb.append(cls)
  boxes = np.array(boxes)
  scores = np.array(scores)
  lb = np.array(lb)
  boxes = boxes[scores >= 0.05].astype(np.int32)
  lb = lb[scores >=float(0.05)]
  scores = scores[scores >=float(0.05)]
  for i in range(len(boxes)):
    j += 1
    x = boxes[i][0]
    y = boxes[i][1]
    w = boxes[i][2]
    h = boxes[i][3]
    imgg = frame[y:y+h, x:x+w]
    try:
      labels.at[j,'image'] = 'img' + str(j)
      labels.at[j,'label'] = lb[i]
      fname = 'img'+str(j)+'.png'
      cv2.imwrite(path+fname,imgg)
    except:
      break

In [43]:
labels.to_csv(path+'labels.csv')